# Data voorbereiden

In [1]:
#Libaries importeren
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

In [2]:
#Lijst maken met alle recepten zonder noten -> csv was al door Martti gemaakt
recepten_df=pd.read_csv("Generated Datasets/Filter_noten_maaltijd_DF.csv")
recepten_zonder_noten = recepten_df.iloc[:,0].tolist()

In [3]:
#Data ophalen
tags = pd.read_csv('/data/foodboost/tags.csv')
ingr = pd.read_csv('/data/foodboost/ingredients.csv')
print("#recepten:", len(ingr.axes[0]))

#recepten: 71809


In [4]:
tags.head()

,Unnamed: 0,recipe,tag
0,0,Kruidnoten met choco-discodip,hollands
1,1,Kruidnoten met choco-discodip,gebak
2,2,Kruidnoten met choco-discodip,gebak
3,3,Kruidnoten met choco-discodip,sinterklaas
4,4,Kruidnoten met choco-discodip,sinterklaasavond


## Filteren op maaltijd (lunch en diner) en noten

In [5]:
#toegestaande tags
tagsmaal= ['lunch','diner','hoofdgerecht']
#filteren op die tags en elk recept 
recepten_lunch_diner = tags[tags['tag'].str.contains('|'.join(tagsmaal))]
recepten_lunch_diner = list(recepten_lunch_diner["recipe"].unique())

#recepten zowel zonder noten als de juiste maaltijd
recepten_zonder_noten_lunch_diner = [value for value in recepten_lunch_diner if value in recepten_zonder_noten]
print("#recepten:", len(recepten_zonder_noten_lunch_diner))

#recepten: 4593


## Ingrediënten pivot tabel maken

In [6]:
#Een pivot tabel maken 
ingr_pivot = ingr.pivot_table(values="recipe", index= ingr.recipe, columns="ingredient", aggfunc="count")
ingr_pivot= ingr_pivot.fillna(0) 

print("#recepten:", len(ingr_pivot.axes[0]))
print("#ingrediënten:", len(ingr_pivot.axes[1]))

#recepten: 8706
#ingrediënten: 7204


In [7]:
#Ingrediënten van de recepten zonder noten en alleen lunch en diner
ingr_pivot = ingr_pivot.loc[recepten_zonder_noten_lunch_diner]

print("#recepten:", len(ingr_pivot.axes[0]))

#recepten: 4593


In [8]:
ingr_pivot.head(3)

ingredient,&amp;samhoud Tomeato gehakt,(Belgische) mayonaise,(arachide)olie,(herten)biefstukjes,(herten)biefstukken,(houdbare) melk,(magere) shoarmareepjes,(olijf)olie,(scharrel)braadworst,(wilde) perzik,...,zwarte rijst,zwarte sesamzaadje,zwarte tagliatelle,zwarte thee,zwarte-peperkorrel,zwarte-peperkorrels,zwartekersenjam,zwarteolijvenplakjes,zwartepeperkorrel,zwijnsteak
recipe,,,,,,,,,,,,,,,,,,,,,
Tartiflette,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ajam pangang,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bloemkoolschotel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Dezelfde ingrediënten samenvoegen als ze te veel op elkaar lijken

In [9]:
#Variabele defineren
ingr_pivot_columnnames= ingr_pivot.columns.values.tolist()
lengte= len(ingr_pivot_columnnames)
dubbele_ingr= []
grens_ratio= 85
ingr_pivot_copy= ingr_pivot.copy()

In [10]:
#Dubbele kolomnamen zoeken
for i in range(lengte):
    if i < (lengte-1):
        Ratio = fuzz.token_sort_ratio(ingr_pivot_columnnames[i],ingr_pivot_columnnames[i+1])
        if Ratio > grens_ratio:
            dubbele_ingr.append((ingr_pivot_columnnames[i], ingr_pivot_columnnames[i+1]))

In [11]:
#Dubbele kolommen samenvoegen en de extra verwijderen
ingr_pivot_copy = ingr_pivot.copy()
for i in dubbele_ingr:
    ingr_pivot_copy[i[1]] = ingr_pivot_copy[i[0]] + ingr_pivot_copy[i[1]]
    ingr_pivot_copy = ingr_pivot_copy.drop([i[0]], axis=1)
    
print("#recepten:", len(ingr_pivot_copy.axes[0]))
print("#ingrediënten:", len(ingr_pivot_copy.axes[1]))

#recepten: 4593
#ingrediënten: 6774


## Filter op ingrediënten die meer dan 40x voorkomen

In [12]:
ingr_pivot_copy = ingr_pivot_copy.loc[:,ingr_pivot_copy.sum()>40]
#Alleen recepten die nog minimaal 1 ingrediënt hebben
ingr_pivot_copy = ingr_pivot_copy.loc[ingr_pivot_copy.sum(axis=1)!=0,:]

print("#recepten:", len(ingr_pivot_copy.axes[0]))
print("#ingrediënten:", len(ingr_pivot_copy.axes[1]))

#recepten: 4535
#ingrediënten: 167


# Output

In [13]:
print("#recepten:", len(ingr_pivot_copy.axes[0]))
print("#ingrediënten:", len(ingr_pivot_copy.axes[1]))
ingr_pivot_copy.head(3)

#recepten: 4535
#ingrediënten: 167


ingredient,(arachide)olie,(olijf)olie,Franse mosterd,Griekse yoghurt,Japanse sojasaus,Parmezaanse kaas,Parmigiano Reggiano,Pecorino Romano (kaas),Tasty Tom trostomaat,aardappelen,...,witlof,witte kaas 45+,witte rijst,witte wijnazijn,zilvervliesrijst,zoete aardappel,zoete chilisaus,zoete puntpaprika's,zonnebloemolie,zout
recipe,,,,,,,,,,,,,,,,,,,,,
Tartiflette,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ajam pangang,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Bloemkoolschotel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#bestand opslaan
ingr_pivot_copy.to_csv('ingr_pivot_lunch_diner_noten.csv')